In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.compose import ColumnTransformer

In [2]:
df = pd.read_csv("covid_toy.csv")
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [3]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, :5], df.iloc[:,-1], test_size=0.1)

In [6]:
X_train.shape, X_test.shape

((90, 5), (10, 5))

# 1. Without using Column Transformer

In [7]:
# adding simple imputer to fill empty cells with simple strategy (Only apply to that column which have Null Values)
si = SimpleImputer()
si.fit(X_train[['fever']])

X_train_fever = si.transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])

X_train_fever.shape

(90, 1)

In [8]:
# ordinal encoding on cough
oe = OrdinalEncoder(categories=[['Mild', 'Strong']])

oe.fit(X_train[['cough']])

X_train_cough = oe.transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])

X_train_cough.shape

(90, 1)

In [9]:
# onehot encoding on gender, city
ohe = OneHotEncoder(drop='first', sparse_output=False, dtype=int)

ohe.fit(X_train[['gender', 'city']])

X_train_gender_city = ohe.transform(X_train[['gender', 'city']])
X_test_gender_city = ohe.transform(X_test[['gender', 'city']])

X_train_gender_city.shape

(90, 4)

In [10]:
# extracting age for concatination
X_train_age = X_train.drop(columns=['gender', 'fever', 'cough', 'city']).values
X_test_age = X_test.drop(columns=['gender', 'fever', 'cough', 'city']).values

X_train_age.shape, X_test_age.shape

((90, 1), (10, 1))

In [11]:
# concatenate all the dataFrame columns
X_train_transform = np.concatenate((X_train_age, X_train_gender_city, X_train_fever, X_train_cough), axis=1)

# also for test
X_test_transform = np.concatenate((X_test_age, X_test_gender_city, X_test_fever, X_test_cough), axis=1)

X_train_transform.shape

(90, 7)

# Using Column Transformer

In [12]:
transformer = ColumnTransformer(transformers=[
    ('tnf1', SimpleImputer(), ['fever']),
    ('tnf2', OneHotEncoder(drop='first', sparse_output=False), ['gender', 'city']),
    ('tnf3', OrdinalEncoder(categories=[['Mild', 'Strong']]), ['cough'])
    
], remainder='passthrough')
# remainder: Specifies what to do with the remaining columns that are not explicitly transformed. It can be set to:
        # -> 'drop' (default): Drops the remaining columns.
        # -> 'passthrough': Passes through the remaining columns without any transformation.

In [13]:
transformer.fit(X_train)

X_train_transform1 = transformer.transform(X_train)
X_test_transform1 = transformer.transform(X_test)

X_train_transform1.shape

(90, 7)

In [14]:
df.head(2)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
